<a href="https://colab.research.google.com/github/RafalDoroz/ai/blob/main/tr/transferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import zipfile

In [ ]:
# Import pretrenowanego modelu ResNet50 (bez warstw wierzchołkowych)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Zamrożenie warstw bazowych, aby nie były trenowane
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# Dodanie własnych warstw na wierzch pretrenowanego modelu
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
output = Dense(2, activation='softmax')(x)  # 2 klasy (np. koty i psy)

# Połączenie bazowego modelu z nowymi warstwami
model = Model(inputs=base_model.input, outputs=output)

# Kompilacja modelu
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Ścieżki
zip_file_path = 'catsdogs.zip'  # Ścieżka do pliku ZIP w katalogu dataset
extract_to_dir = 'dataset'            # Katalog, do którego rozpakowujemy

# Rozpakowanie archiwum ZIP do katalogu dataset
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_dir)
print(f"Zawartość pliku {zip_file_path} została rozpakowana do katalogu {extract_to_dir}")

# Przygotowanie generatora obrazów
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',  # Ścieżka do danych treningowych
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    'dataset/validation',  # Ścieżka do danych walidacyjnych
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Zawartość pliku catsdogs.zip została rozpakowana do katalogu dataset
Found 40 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [ ]:
# Trenowanie modelu
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


Epoch 1/10


In [ ]:
# Wizualizacja dokładności treningu i walidacji
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Wizualizacja straty treningu i walidacji
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Ocena modelu na zbiorze walidacyjnym
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
